# Requirement

* anaconda3-4.3.1

# Import

In [1]:
%matplotlib inline
import glob
from IPython.display import display
import numpy as np
import pandas as pd
import re
import socket
from sklearn import datasets
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

/Users/nomiyama/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Define

In [2]:
file_name = '../Processing/Recorder/data/*.csv'
sensing_num = 4
frequency_num = (255 - 30) // sensing_num
client_host = '192.168.1.27'
client_port = 10001
server_host = ''
server_port = 10002

# Preprocess

In [3]:
files =  glob.glob(file_name)
voltages = [None for f in files]
for i, f in enumerate(files):
    column_names = ['label{0}'.format(i)]
    column_names.extend(['frequency{0}_{1:02d}'.format(i, j+1) for j in range(frequency_num)])
    voltages[i] = pd.read_csv(f, names=column_names, index_col=False)
    voltages[i] = voltages[i].fillna(method='ffill', axis=1)

voltage = pd.concat(voltages, axis=1)

# inspect consistency
for i in range(sensing_num - 1):
    voltage = voltage[voltage['label{0}'.format(i)] == voltage['label{0}'.format(i + 1)]]

y = voltage['label0']
for i in range(sensing_num):
    voltage = voltage.drop('label{0}'.format(i), axis=1)
X = voltage

# Learn
* ovo = One vs One
* ovr = One vs the Rest

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None )
ovo_model = SVC(kernel='linear', random_state=None)
ovr_model = OneVsRestClassifier(ovo_model)
ovo_model.fit(X_train, y_train)
ovr_model.fit(X_train, y_train)

ovo_prediction = ovo_model.predict(X_test)
accuracy_test = accuracy_score(y_test, ovo_prediction)
print('正解率： %.2f (One vs One)' % accuracy_test)

ovr_prediction = ovr_model.predict(X_test)
accuracy_test = accuracy_score(y_test, ovr_prediction)
print('正解率： %.2f (One vs the Rest)' % accuracy_test)

正解率： 1.00 (One vs One)
正解率： 1.00 (One vs the Rest)


# Real Time Prediction
* python側を先に起動
* Processingのserver.ip()を確認

In [7]:
pattern = re.compile(r'.*finished$')

string_data = ''
voltage = pd.Series([np.nan for i in range(frequency_num * sensing_num)])
voltages = [pd.Series([np.nan for i in range(frequency_num)]) for i in range(sensing_num)]

try:
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((server_host, server_port))
        s.listen(1)
        connection, address = s.accept()
        with connection:
            print('Connected by', address)
            
            while True:
                response = connection.recv(1024)
                if not response:
                    continue
                string_data += response.decode()
                if (re.match(pattern, string_data)):
                    data_list = [x.split('/')[1:] for x in string_data.split('/pause/')[:sensing_num]]
                    for i, data in enumerate(data_list):
                        data = [float(x) for x in data]    
                        voltages[i][:len(data)] = data[-1 * frequency_num:]
                        voltages[i] = voltages[i].fillna(method='ffill')

                    voltage[:] = pd.concat(voltages)[:]
                    state = ovo_model.predict([voltage])[0]

                    socket_client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                    socket_client.connect((client_host, client_port))
                    socket_client.send(state.encode('utf-8'))

                    string_data=''
                    voltage = pd.Series([np.nan for i in range(frequency_num * sensing_num)])
                    voltages = [pd.Series([np.nan for i in range(frequency_num)]) for i in range(sensing_num)]

except KeyboardInterrupt:
    print('Exit')

Connected by ('127.0.0.1', 65305)
Exit
